In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

#로그인 데이터, 팀 생성 데이터 추가
df_login = pd.read_csv("/Users/jiseokchang/Desktop/2020_login.txt", sep = "\t")
df_team = pd.read_csv("/Users/jiseokchang/Desktop/team.txt", sep = "\t")

df = pd.merge(df_team, df_login, on = "team_id", how = "left")

#datetime 타입 변환
df["crt_date"] = pd.to_datetime(df["crt_date"])
df["login_date"] = pd.to_datetime(df["login_date"])
df["crt_date"] = df["crt_date"].apply(lambda x: x.strftime("%Y-%m-%d"))
df["login_date"] = df["login_date"].apply(lambda x: x.strftime("%Y-%m-%d"))

#접속 여부 유니크 정리(1일 1회 최대)
df_unique = df.drop_duplicates(['team_id', 'login_date'], keep = "first")
daily_new_user = df_unique.groupby("crt_date").agg({"team_id":"nunique"})

#계정 생성일 확인 칼럼 생성
df_unique["is_first"] = df_unique.apply(lambda x: True if x["login_date"] == x["crt_date"] else False, axis = 1)
date_list = pd.date_range(start='2020-05-05', end='2020-06-11')
date_list = date_list.strftime("%Y-%m-%d")

#리텐션 계산 함수 정의
def daily_retention(df, date):
    cohort = df_unique[(df_unique["is_first"] == True) & (df["crt_date"] == date)]
    num_of_users = cohort["team_id"].nunique()
    print("Number of unique users on %s:" %date, num_of_users)
    
    list_cohort= cohort['team_id'].unique()
    df_cohort= df[df['team_id'].isin(list_cohort)]
    cohort_data=pd.DataFrame((df_cohort.groupby(['login_date']).nunique()['team_id']/num_of_users*100).round(2))
    return cohort_data

<ipython-input-1-9748f118c0d9>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["is_first"] = df_unique.apply(lambda x: True if x["login_date"] == x["crt_date"] else False, axis = 1)


In [2]:
final_df = pd.DataFrame(index = date_list)

#for문으로 일자를 돌며 기준 일자별 리텐션 계산
for index in final_df.index:
    temp = daily_retention(df_unique, index)
    final_df[index] = temp

Number of unique users on 2020-05-05: 526
Number of unique users on 2020-05-06: 493
Number of unique users on 2020-05-07: 556
Number of unique users on 2020-05-08: 532
Number of unique users on 2020-05-09: 719
Number of unique users on 2020-05-10: 787
Number of unique users on 2020-05-11: 611
Number of unique users on 2020-05-12: 717
Number of unique users on 2020-05-13: 738
Number of unique users on 2020-05-14: 763
Number of unique users on 2020-05-15: 811
Number of unique users on 2020-05-16: 965
Number of unique users on 2020-05-17: 911
Number of unique users on 2020-05-18: 839
Number of unique users on 2020-05-19: 745
Number of unique users on 2020-05-20: 685
Number of unique users on 2020-05-21: 717
Number of unique users on 2020-05-22: 583
Number of unique users on 2020-05-23: 687
Number of unique users on 2020-05-24: 827
Number of unique users on 2020-05-25: 627
Number of unique users on 2020-05-26: 655
Number of unique users on 2020-05-27: 627
Number of unique users on 2020-05-

In [3]:
final_df.to_csv("/Users/jiseokchang/Desktop/2020_retention.csv", sep = ",")